This Python script simulates the dynamics of charged particles under relativistic conditions using the Pygame library. It is designed to visualize and explore concepts such as electromagnetic fields, synchrotron radiation, and the effects of relativity on particle interactions. Here's a breakdown of what the script does:

### Imports and Constants
- **Pygame** is used for graphical rendering and interaction.
- **Numpy** is utilized for numerical operations, especially in vector and matrix computations.
- Constants like `WIDTH`, `HEIGHT`, `SPEED_OF_LIGHT`, and `ELEMENTARY_CHARGE` are defined for use in simulations.

### Synchrotron Function
- `synchrotron_function(x)`: Approximates a function related to synchrotron radiation, which particles emit when they are accelerated in magnetic fields.

### Particle Class
- Represents charged particles with properties like position (`x`, `y`, `z`), `charge`, `mass`, and `velocity`.
- `update`: Updates particle's state based on electromagnetic fields and other particles. It calculates forces from given electric and magnetic fields and updates the particle's velocity and position considering relativistic effects (like mass increase at speeds close to the speed of light and time dilation).
- `synchrotron_radiation_spectrum` and `synchrotron_radiation_power`: Calculate the spectrum and power of synchrotron radiation emitted by the particle.
- `retarded_position` and `retarded_fields`: Compute the position and electromagnetic fields of the particle at previous times, accounting for the finite speed of light (this is part of what makes the simulation 'relativistic').
- `relativistic_mass` and `time_dilation`: Compute the relativistic mass and time dilation effects based on the particle’s velocity.

### ParticleManager Class
- Manages a collection of `Particle` instances.
- `add_particle`: Adds a new particle to the simulation.
- `draw`: Draws all particles on the Pygame screen.
- `update_interactions`: Updates interactions between all particles, considering retarded fields, which simulate the delay in the effect of one particle's field on another due to the finite speed of light.
- `update`: Updates all particles’ states and interactions between them.

### Field Functions
- `electric_field` and `magnetic_field`: Functions to compute the electric and magnetic fields at any point in space based on predefined distributions or configurations.

### Simulation Loop
- Initializes Pygame and sets up the simulation environment.
- Enters a loop where it processes user events (like quitting or adding particles with mouse clicks), updates the state of the simulation (particle movements and interactions), and renders the updated state to the screen.
- Uses Pygame’s drawing functions to visualize particles and possibly field lines or intensity.

### Key Concepts Illustrated by the Script
- **Relativistic Effects**: The script accounts for effects such as time dilation and relativistic mass increase, essential when simulating particles moving at speeds close to the speed of light.
- **Electromagnetic Interactions**: It computes interactions between particles based on electromagnetic fields, including those generated by the particles themselves, considering the retardation effect due to the finite speed of light.
- **Synchrotron Radiation**: It calculates and possibly visualizes the radiation emitted by charged particles when accelerated, which is significant in many high-energy physics contexts.

Overall, this script provides a platform for visualizing and understanding the complex dynamics involved in relativistic particle interactions, including electromagnetic forces and radiation. It serves as an educational tool for illustrating concepts from electromagnetism and relativistic physics.

In [1]:
# Acceleration  +Momentum 
# V 9.0.1 Relativistic Particle Simulation
 #conservation of ebergy
# This version of the simulation focuses on the dynamic behavior of relativistic 
# particles under various electromagnetic influences. Below is an overview of the
# key features and physical phenomena modeled in the code:

### Particle Characteristics
# Lorentz Force**: Accurately models the electromagnetic force on charged particles, 
# integrating both electric and magnetic field effects.

### Energy Loss and Radiation
# Synchrotron Radiation**: Simulates energy dissipation due to synchrotron radiation 
# for particles in magnetic fields. This involves computing critical frequencies and the 
# associated power radiated.

### Advanced Electromagnetic Effects
# Landau-Lifshitz Radiation Reaction**: Incorporates a simplified version of the 
# Landau-Lifshitz expression to account for the radiation reaction force alongside 
# the Lorentz force, enhancing the realism in modeling particle dynamics.

# Retarded Time Effects**: Takes into account the finite speed of light for interactions 
# between particles, employing the concept of retarded time to calculate forces more accurately.

# Liénard-Wiechert Potentials**: Utilizes electric forces between moving particles 
# to model retarded effects based on Liénard-Wiechert potentials.

# Biot-Savart Law**: Applies the Biot-Savart law for calculating magnetic forces 
# between moving particles, further refining the simulation of retarded effects.

### Numerical Integration and Relativistic Modeling
# Runge-Kutta 4th Order Integration (RK4)**: Employs the RK4 method for robust 
# numerical integration, updating particle positions and velocities over time 
# considering the applied forces.

# Relativistic Effects**: Ensures the simulation remains physically accurate 
# at high velocities by incorporating relativistic corrections, notably through 
# the use of the Lorentz factor (gamma) in force, acceleration, and energy calculations.

### Simulation Enhancements
# Lorentz Transformation of Fields**: Includes transformations between the 
# laboratory frame of reference and the particle's frame of reference, allowing 
# for detailed analysis of relativistic effects.
#  Adaptive Time Stepping**: Adjusts the time step dynamically based on the 
# simulation conditions to optimize accuracy and computational efficiency.
# Relativistic Equations of Motion**: Fully integrates relativistic equations 
# of motion into the simulation framework, ensuring comprehensive coverage of relativistic dynamics.

 

import math
import pygame
import random
import numpy as np
from pygame import surfarray
import colorsys
from physics import ElectromagneticEquations
import uuid

class Particle:      
    def __init__(self, initial_pos, initial_velocity, 
                 charge, mass, force_method, radiation_reaction, velocity_method,
                 electric_field_function, magnetic_field_function):          
        self.c = 299792458                   
        constants = {       }
        self.force_method = lambda pos, vel: force_method(self, pos, vel)
        self.rad_method = radiation_reaction   
        self.velocity_method = velocity_method
        self.id = str(uuid.uuid4())  
        self.sync_freq = 0 
        self.charge, self.mass = charge, mass 
        self.dt = 1e-12
        self.total_E_field = np.array([0.0, 0.0, 0.0])
        self.total_B_field = np.array([0.0, 0.0, 0.0]) 
        self.thickness = 0
        self.position = initial_pos
        self.velocity = em_equations.limit_speed(initial_velocity)
        self.acc = np.array([0.0, 0.0, 0.0])         
        self.trajectory=25e2
        self.dt_traj  = [self.dt]
        self.pos_traj = [self.position]
        self.vel_traj = [self.velocity]
        self.acc_traj = [self.acc]  
         
        self.gamma = em_equations.Gamma(self.velocity)        
        self.total_energy =self.gamma * self.mass * (self.c ** 2)  
        self.energy_loss = 0  
        self.electric_function = electric_field_function
        self.magnetic_function = magnetic_field_function
        self.radius = 5  
        self.color = (0, 0, 0)               
        
    def update(self, particles, dt):
        self.dt = dt
        # Calculate base and retarded electric and magnetic fields                
        retarded_E_field, retarded_B_field = self.retarded_effects(particles, dt)          
        self.total_E_field =  self.electric_function(self.position)  + retarded_E_field
        self.total_B_field =  self.magnetic_function(self.position)  + retarded_B_field  
        
        ##########################################################################
        #Energy loss due to radiation          
        # v_synch_new, delta_Es, KE_final = em_equations.synchrotron_radiation(self, self.total_B_field, self.velocity, dt)        
        # velocety correction due to radiation_reaction
        # v_rad_new, delta_Er, KE_final = self.rad_method(self, self.total_B_field, self.velocity, dt)         
        # v_new = self.update_velocity(v_synch_new , v_rad_new) #v_synch_new                
        ###########################################################################
        
        # Momentum before the update 
        Energy_old =self.gamma * self.mass * (self.c ** 2) 
        P_old = self.gamma*self.mass*self.velocity  
        
        self.position, self.velocity = self.velocity_method(self)   
        self.velocity = em_equations.limit_speed(self.velocity)     
        #print
        self.gamma = em_equations.Gamma(self.velocity)
        P_new = self.gamma*self.mass*self.velocity
        self.acc = (P_new-P_old)/self.dt 

        self.total_energy = self.gamma * self.mass * (self.c ** 2)                                
        self.energy_loss = Energy_old - self.total_energy      
        self.update_color_based_on_radiation()      
        
        # Update Trajectory         
        self.step_count = getattr(self, 'step_count', 0) + 1        
        attributes = ('position', 'velocity', 'dt', 'acc')
        for attr, traj in zip(attributes, (self.pos_traj, self.vel_traj, self.dt_traj, self.acc_traj)):
            traj.append(np.copy(getattr(self, attr)))
            if self.step_count <= 100 or len(traj) > self.trajectory:
                traj.pop(0)  

    def update_error(self, particles, dt):
        self.dt = dt
        # Calculate base and retarded electric and magnetic fields                
        retarded_E_field, retarded_B_field = self.retarded_effects(particles, dt)          
        self.total_E_field = self.electric_function(self.position) + retarded_E_field
        self.total_B_field = self.magnetic_function(self.position) + retarded_B_field        
        # Momentum before the update 
        P_old = self.gamma*self.mass*self.velocity
        self.position, self.velocity = self.velocity_method(self)
        #self.velocity = em_equations.limit_speed(self.velocity)                        
        #Momentum after the update
        gamma = em_equations.Gamma(self.velocity)  
        P_new = gamma*self.mass*self.velocity
        self.acc = (P_new-P_old)/self.dt                     

    def update_velocity(self, v_synch_new, v_rad_new):
        # Directly calculate the changes in velocity due to synchrotron and radiation reaction
        delta_v_synch = np.linalg.norm(v_synch_new - self.velocity)
        delta_v_rad = np.linalg.norm(v_rad_new - self.velocity)
        # Normalize the magnetic field and current velocity for direction calculations
        B_unit = self.total_B_field / np.linalg.norm(self.total_B_field)
        v_unit = self.velocity / np.linalg.norm(self.velocity)
        # Calculate the direction perpendicular to B_field for synchrotron effect
        perp_dir = np.cross(np.cross(B_unit, v_unit), B_unit)
        perp_dir /= np.linalg.norm(perp_dir)  # Ensure it's a unit vector
        # Update the velocity by applying changes in the calculated directions
        #velocity =self.velocity + delta_v_synch * perp_dir + delta_v_rad * v_unit
        return self.velocity + delta_v_synch * perp_dir + delta_v_rad * v_unit
            
    def retarded_effects(self, particles, dt):                      
        total_E, total_B = np.zeros(3), np.zeros(3)        
        for p in particles:
            if  p.id == self.id:
                continue            
            r, retarded_time = em_equations.calculate_retardation(self, p)
            retarded_position, retarded_velocity, retarded_acc = em_equations.retarded_state(p, retarded_time)
            r_retarded, r_retarded_mag, r_retarded_unit = em_equations.calculate_retarded_distance(self, retarded_position)            
            E = em_equations.calculate_electric_field(p, r_retarded, r_retarded_mag, r_retarded_unit, retarded_velocity, retarded_acc)
            B = em_equations.calculate_magnetic_field(p, r_retarded_mag, r_retarded_unit, retarded_velocity)            
            total_E += E
            total_B += B
        return total_E, total_B              

###################################################################################            
    def update_color_based_on_radiation(self):
        max_power = 1e3 * self.mass * self.c**2
        normalized_power = min(self.energy_loss / self.total_energy, 1)
        brightness = np.clip(np.log10((0.7 + 0.3 * normalized_power) * 10), 0, 1)

        normalized_speed = np.clip(np.log10(10 * np.linalg.norm(em_equations.limit_speed(self.velocity)) / self.c), 0, 1)
        hue = 0.3 * (1 - normalized_speed)  # Adjusted to ensure hue is within the [0, 1] range

        min_gamma, max_gamma = 1, 30
        normalized_gamma = (self.gamma - min_gamma) / (max_gamma - min_gamma)
        saturation = 0.5 + 0.5 * min(normalized_gamma, 0.99)

        # Ensure the velocity is less than c before calculating color
        if np.linalg.norm(self.velocity) < self.c:
            self.color = tuple(int(255 * component) for component in colorsys.hsv_to_rgb(hue, saturation, brightness))
        else:
            pass   

    
######################################################################################
#############################################################################

class ParticleManager:
    def __init__(self):
        self.particles = []  # List to hold all particles
        self.dt = dt
        self.c = 299792458  
        self.ERROR_THRESHOLD = 0.01
        self.MIN_DT = 1e-15
        self.MAX_DT = 1e-8
        self.MIN_INCREASE_FACTOR = 1.05
        self.MAX_DECREASE_FACTOR = 0.85
    def update(self, electric_field, magnetic_field):
        # Initial factors set based on class or instance level preferences
        decrease_factor = 0.7
        increase_factor = 1.2

        for particle in self.particles:
            error = self.estimate_error(particle, electric_field, magnetic_field)
            # Proceed with adjusting timestep only if error != 1
            if error != 0 and error > 0.006:
                self.adjust_timestep(error, self.ERROR_THRESHOLD, decrease_factor, increase_factor, self.MIN_DT, self.MAX_DT)
                # Adjust factors based on operation outcome
                decrease_factor = max(decrease_factor * 0.9, self.MAX_DECREASE_FACTOR)
                increase_factor = max(increase_factor * 0.975, self.MIN_INCREASE_FACTOR)
        
        self.update_particles()

    def estimate_error(self,p, electric_field, magnetic_field):
        # Save the current state of the simulation
        original_state = self.save_state(p)
        # Perform a single step with Δt
        p.update_error( self.particles, self.dt )
        final_state_single_step = self.save_state(p)
        # Restore the original state
        self.restore_state(original_state,p)
        # Perform two steps with Δt/2
        p.update_error( self.particles, self.dt/2 )
        p.update_error( self.particles, self.dt/2 ) 
        final_state_two_steps = self.save_state(p)
        # Restore the original state again
        self.restore_state(original_state,p)
        # Calculate the error between the two approaches
        #error = self.calculate_difference(final_state_single_step, final_state_two_steps)      
        error =np.linalg.norm(np.array(final_state_single_step['position'])-np.array(final_state_two_steps['position']))
        #print(error)
        return error

    def adjust_timestep(self, error, threshold, decrease_factor, increase_factor, min_dt, max_dt):
        if error > threshold:
            self.dt *= decrease_factor
        else:
            self.dt = min(self.dt * increase_factor, max_dt)
        self.dt = max(min_dt, min(self.dt, max_dt))   
        
    def add_particle(self, initial_pos, initial_velocity, charge, mass,dt, force_method, radiation_reaction, velocity_method,
                     electric_field, magnetic_field):
        # Add a new particle to the list
        self.particles.append(Particle(initial_pos, initial_velocity, charge, mass, 
                                       force_method, radiation_reaction, velocity_method, electric_field, magnetic_field)) 
                
    def update_particles(self):
        for particle in self.particles:
            particle.update( self.particles, self.dt)        
    
    def save_state(self,p):
        return {'position': p.position, 
                'velocity': p.velocity.copy(), 
                'acc': p.acc.copy() }
        
    def restore_state(self, state, p):
        p.position = state['position']
        p.velocity = state['velocity']
        p.acc = state['acc']                 

    def draw_particles(self, screen):
        """Draws all particles on the screen."""
        for particle in self.particles:
            pygame.draw.circle(screen, particle.color, (int(particle.position[0]), int(particle.position[1])), particle.radius, particle.thickness)

    def draw_bar(self, screen, label, value, max_value, position, color=(0, 255, 0)):
        """Draws a labeled bar for displaying metrics like velocity or time dilation."""
        bar_color = (255, 0, 0)  # Red
        text_color = (255, 255, 255)  # White
        bar_width = 20
        bar_height = 100
        # Calculate the logarithm of the value and max_value (add a small constant to avoid taking the log of 0)
        log_value = np.log10(value + epsilon)
        log_max_value = np.log10(max_value + epsilon)
        # Calculate the filled height based on the logarithmic scale
        #print(bar_height , log_value , log_max_value)
        filled_height = int(bar_height * log_value / log_max_value)
        bar_x, bar_y = position
        # Draw the bar outline and filled portion
        pygame.draw.rect(screen, (255, 255, 255), (bar_x, bar_y, bar_width, bar_height), 1)
        pygame.draw.rect(screen, color, (bar_x, bar_y + bar_height - filled_height, bar_width, filled_height))
        # Label the bar
        font = pygame.font.Font(None, 18)
        text = font.render(label, True, (255, 255, 255))
        text_rect = text.get_rect(centerx=bar_x + bar_width // 2, y=bar_y + bar_height + 5)
        screen.blit(text, text_rect)
        # Display the value using scientific notation
        max_value_text = font.render(f"{value:.1e}", True, (255, 0, 0))
        max_value_rect = max_value_text.get_rect(centerx=position[0] + bar_width // 2, bottom=position[1] - bar_height + 80)
        screen.blit(max_value_text, max_value_rect)  
        font = pygame.font.Font(None, 24)
        dt_text = font.render(f"dt: {self.dt:.1e}", True, (255, 255, 255))
        dt_text_rect = dt_text.get_rect(left=10, top=10)
        screen.blit(dt_text, dt_text_rect)
        #dt_text = font.render(f"dt: {self.dt:.1e}", True, (255, 255, 255))
        #dt_text_rect = dt_text.get_rect(left=10, top=10)
        #screen.blit(dt_text, dt_text_rect)
        
    def draw(self, screen):
        """Main drawing function."""                
        MAX_TIME_DILATION = 200  
        MAX_ENERGY = 0.01
        MAX_SYNC = 1e11
        MAX_VELOCITY= self.c 
        for particle in self.particles:
            self.draw_trajectory(screen, particle)
        # Draw all particles
        self.draw_particles(screen)
        # Find the particle with the maximum feature
        max_velocity_particle = max(self.particles, key=lambda p: np.linalg.norm(p.velocity))
        
        velocity = np.linalg.norm(max_velocity_particle.velocity)
        
        gamma = max_velocity_particle.gamma
        #print(max_velocity_particle.energy_loss)
        total_energy = (0.1+max_velocity_particle.energy_loss)/ (self.c ** 2)   
        sync_freq = max_velocity_particle.sync_freq
        
        Max_ENERGY_LOSS = 1e4*max_velocity_particle.mass*self.c**2 
        energy_loss = max_velocity_particle.energy_loss 
    
        #print(energy_loss)
        # Bar positions
        screen_width, screen_height = screen.get_size()
        velocity_bar_position = (screen_width - 100, 160 + (screen_height - 20) // 2)
        time_dilation_bar_position = (velocity_bar_position[0] + 50, velocity_bar_position[1])
        total_energy_bar_position = (velocity_bar_position[0] - 50, velocity_bar_position[1])  
        total_sync_bar_position = (velocity_bar_position[0] - 50, velocity_bar_position[1]) 
        total_energy_loss_position = (velocity_bar_position[0] - 50, velocity_bar_position[1]) 
        
        # Draw bars
        self.draw_bar(screen, "V[c]", velocity, self.c, velocity_bar_position)
        self.draw_bar(screen, "Gamma", gamma, MAX_TIME_DILATION, time_dilation_bar_position)
        #self.draw_bar(screen, "Sync Rad", sync_freq, MAX_SYNC, total_sync_bar_position)
        #print(energy_loss) max(value, 0)
        self.draw_bar(screen, "Enrg Loss", max(energy_loss,0), Max_ENERGY_LOSS, total_energy_loss_position)
   
    def draw_trajectory(self, screen, particle):
        """Draws the trajectory of a single particle."""
        # Assuming particle.trajectory stores a list of past positions [(x1, y1), (x2, y2), ...]
        if hasattr(particle, 'pos_traj') and len(particle.pos_traj) > 1:
            for strt_pos, end_pos in zip(particle.pos_traj[:-1], particle.pos_traj[1:]):                
                pygame.draw.line(screen, particle.color, (int(strt_pos[0]), int(strt_pos[1])), (int(end_pos[0]), int(end_pos[1])))

    def draw_arrows(self, surface, E_trans, B_trans, width, height, grid_spac, scale_fact, E_color, B_color):
        # Meshgrid for screen coordinates
        x_in, y_in = np.meshgrid(np.arange(0, width, grid_spac), np.arange(0, height, grid_spac))
    
        def process_field(field, arrow_scale):
            magnitude = np.linalg.norm(field, axis=0)
            direction = field / (magnitude + epsilon)
            arrow_lengths = np.minimum(magnitude * arrow_scale, 25)  # Clamp arrow lengths to a maximum value
            arrow_end_x = x_in + arrow_lengths * direction[0]
            arrow_end_y = y_in + arrow_lengths * direction[1]
            start_pos = np.column_stack((x_in.ravel(), y_in.ravel()))
            end_pos = np.column_stack((arrow_end_x.ravel(), arrow_end_y.ravel()))
            return start_pos, end_pos, magnitude
    
        # Process electric field
        E_start_pos, E_end_pos, E_magnitude = process_field(E_trans, arrow_scale=20)
    
        # Process magnetic field
        B_start_pos, B_end_pos, B_magnitude = process_field(B_trans, arrow_scale=1e1)
    
        # Calculate the scalar ratio between electric and magnetic fields
        ratio = E_magnitude / B_magnitude
    
        # Print the average scalar ratio
        #print("Average scalar ratio (E/B):", np.mean(ratio))
    
        # Draw the magnetic field arrows first
        for start, end in zip(B_start_pos, B_end_pos):
            pygame.draw.line(surface, B_color, start.astype(int), end.astype(int), 1)
    
        # Draw the electric field arrows on top
        for start, end in zip(E_start_pos, E_end_pos):
            pygame.draw.line(surface, E_color, start.astype(int), end.astype(int), 2)  # Thicker lines for electric field

############################################################################# 

def electric_field(position):
    EPSILON_0 = 8.8541878128e-12
    x, y, z =position
    # _chaos  Constants     
    k = 1 / (4 * np.pi * EPSILON_0)
    Q = Q_F# 1e-14# Q_F Q#0e-13#-2#1e17*ELEMENTARY_CHARGE
    source_x, source_y, source_z = 401, 300, 0  # Position of the center of the distribution
    sigma_x, sigma_y, sigma_z = 160, 120, 50  # Standard deviations of the distribution
    Lx, Ly = WIDTH, HEIGHT  # Dimensions of the simulation domain
    # Small value to avoid division by zero
    # Compute differences in coordinates
    dx, dy, dz = x - source_x, y - source_y, z - source_z
    # Compute the Gaussian distribution
    gaussian = Q * np.exp(-0.5 * ((dx/sigma_x)**2 + (dy/sigma_y)**2 + (dz/sigma_z)**2))
    # Compute the chaotic perturbation
    perturbation = 0.1 * (np.sin(2 * np.pi * 2 * x / Lx - z) + 0.5 * np.cos(3 * np.pi * 6 * y / Ly - 2*z) + 0.2 * np.sin(x*y*z))
    # Combine the Gaussian distribution and chaotic perturbation
    field_magnitude = gaussian #* (1 + 2*perturbation)
    #field_magnitude =   perturbation
    # Compute the electric field components
    field_x = k * field_magnitude * dx / (sigma_x**2 )
    field_y = k * field_magnitude * dy / (sigma_y**2 )
    field_z = k * field_magnitude * dz / (sigma_z**2 )
    return np.array([field_x, field_y, field_z])
         
def magnetic_field(position):
    x, y, z =position
    # Define the magnetic field components based on your requirements
    B_x = 0#np.zeros_like(x)
    B_y =  0#np.zeros_like(y)
    B_z = B#np.ones_like(z) * 1e-1 # Example: constant magnetic field along the z-axis    
    return np.array([B_x, B_y, B_z ])

def electric_field_point(x, y, z, charge_pos):
    EPSILON_0 = 8.8541878128e-12 
    k = 1 / (4 * np.pi * EPSILON_0)
    Q =0.5  # Charge of the point charge
    source_x, source_y, source_z = charge_pos  # Unpack the charge position
    dx, dy, dz = x - source_x, y - source_y, z - source_z
    distance_squared = dx**2 + dy**2 + dz**2
    field_magnitude = k * Q / distance_squared  # Coulomb's Law
    # Ensure we don't divide by zero
    distance = np.sqrt(distance_squared)
    distance = np.maximum(distance, epsilon)  # A small value to avoid division by zero    
    field_x = field_magnitude * dx / distance
    field_y = field_magnitude * dy / distance
    field_z = field_magnitude * dz / distance
    return np.array([field_x, field_y, field_z])

def magnetic_field_(x, y, z):
    # Define the magnetic field components based on your requirements
    B_x = np.zeros_like(x)
    B_y = np.zeros_like(y)
    B_z = np.ones_like(z) * 1e-1 # Example: constant magnetic field along the z-axis    
    return np.stack((B_x, B_y, B_z), axis=0)

def electric_field_theoretical(x, y, z):
    k = 8.9875517923e9  # Coulomb's constant
    r = np.sqrt(x**2 + y**2 + z**2)
    E_magnitude = k * q / r**2  # Replace q with the source charge
    E_direction = np.array([x, y, z]) / r  # Unit vector in the direction of r
    return E_magnitude * E_direction

def electric_field_acc(x, y, z):
    EPSILON_0 = 8.8541878128e-12 
    # Constants for the accelerator
    center_x, center_y, center_z = 400, 300, 0  # Central point of the accelerator
    Q = 1  # Charge creating the field, simplified assumption
    k = 1 / (4 * np.pi * EPSILON_0)  # Coulomb's constant
    # Compute differences from the center and distances
    dx, dy, dz = x - center_x, y - center_y, z - center_z
    distance = np.sqrt(dx**2 + dy**2 + dz**2) + epsilon  # Add epsilon to avoid division by zero
    # Compute radial and angular directions
    radial_direction = np.array([dx, dy, dz]) / distance
    angular_direction = np.array([-dy, dx, np.zeros_like(dz)]) / (np.sqrt(dx**2 + dy**2) + epsilon)  
    # Compute the electric field
    electric_field = Q * k * radial_direction  # Simplified; adjust as needed for your model
    return electric_field

def electric_field_4(x, y, z):
    # Define the electric field for acceleration
    ELECTRIC_FIELD_STRENGTH=10
    ACCELERATO_LENGTH = 10
    if 0 <= z < ACCELERATO_LENGTH:
        return np.array([0, 0, ELECTRIC_FIELD_STRENGTH])  # Constant electric field along the z-axis
    else:
        return np.array([0, 0, 0])  # No electric field outside the accelerator

def magnetic_field_g(x, y, z):
    MAGNETIC_FIELD_STRENGTH = 1
    ACCELERATOR_LENGTH = 100
    # Convert scalar inputs to arrays
    if isinstance(z, (int, float)):
        z = np.array([z])
    # Define the magnetic field for guiding the particles
    mask = (0 <= z) & (z < ACCELERATOR_LENGTH)
    B = np.zeros((3, *z.shape))
    B[1, mask] = MAGNETIC_FIELD_STRENGTH
    # If the input was scalar, return a 1D array
    if B.shape[1] == 1:
        return B[:, 0]
    return B

def electric_field_spiral (x, y, z):
    # Constants
    k = 8.9875517873681764e9
    Q = 1
    source_x, source_y, source_z = 400, 300, 0  # Position of the center of the distribution
    sigma_x, sigma_y, sigma_z = 50, 50, 50  # Standard deviations of the distribution
    # Compute differences in coordinates
    dx, dy, dz = x - source_x, y - source_y, z - source_z
    # Compute the radial distance and angle in the xy-plane
    r = np.sqrt(dx**2 + dy**2)
    theta = np.arctan2(dy, dx)
    # Compute the spiral factor
    spiral_factor = np.exp(1j * (r / 20))
    # Compute the Gaussian distribution
    gaussian = Q * np.exp(-0.5 * ((dx/sigma_x)**2 + (dy/sigma_y)**2 + (dz/sigma_z)**2))
    # Compute the electric field components
    field_x = k * gaussian * (dx / (sigma_x**2 + epsilon) * spiral_factor.real - dy / (sigma_y**2 + epsilon) * spiral_factor.imag)
    field_y = k * gaussian * (dy / (sigma_y**2 + epsilon) * spiral_factor.real + dx / (sigma_x**2 + epsilon) * spiral_factor.imag)
    field_z = k * gaussian * dz / (sigma_z**2 + epsilon)
    return np.array([field_x, field_y, field_z])
 
#############################################################################
# Constants
from itertools import cycle
ARROW_COLOR_E = (128, 128, 128)
ARROW_COLOR_B = (128, 0, 0)
ARROW_WIDTH = 1
SCALE_FACTOR = 1
GRID_SPACING = 20
WIDTH, HEIGHT = 800, 600
epsilon = np.finfo(np.float64).eps

# Physical constants
ELEMENTARY_CHARGE = 1.6*10e-19
ERROR_THRESHOLD = 0.01
dt = 1e-8
B =0.09 
Q_F= 0#+1e17 * ELEMENTARY_CHARGE 
num_prticle =3
# Particle properties

multiply = 3e4
e_mass = 9.10*1e-31 * multiply 
p_mass = 1.67*1e-27 * multiply

initial_velocity = np.array([1e6, 0.0, 0.0])
e_initial_velocity = np.array([2e8, 0.0, 0.0])
e_charge = -ELEMENTARY_CHARGE 
p_charge = ELEMENTARY_CHARGE

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Particle Simulation")
clock = pygame.time.Clock()
electric_field_surface = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)

# Electromagnetic equations and particle manager
constants = {}
em_equations = ElectromagneticEquations(constants)
particle_manager = ParticleManager()

# Electric and magnetic field grid
x_in, y_in = np.meshgrid(range(0, WIDTH, GRID_SPACING), range(0, HEIGHT, GRID_SPACING))
E_field = electric_field((x_in, y_in, 0))
B_field = magnetic_field((x_in, y_in, 0))
# Initialize transformed fields
E_transformed = E_field
B_transformed = B_field

# Add initial particle
particle_manager.add_particle(np.array([401.0, 300.0, 0.0]), 
                              initial_velocity,  
                              - p_charge,  
                              p_mass,
                              dt, 
                              em_equations.Lorentz, 
                              em_equations.radiation_reaction,
                              em_equations.boris_push_single,  
                              electric_field,  
                              magnetic_field)

counter1 = 0
counter = 0
running = True

while running:
    #number_of_particles  = 3
    #counter1 += 1

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if len(particle_manager.particles) < num_prticle:
                x, y = pygame.mouse.get_pos()
                method_names_and_thicknesses = [("boris_push_single", 1), ("rk6_step", 2)]
                method_name, thickness = method_names_and_thicknesses[len(particle_manager.particles) % len(method_names_and_thicknesses)]
                push_method = getattr(em_equations, method_name)
                
                particle_manager.add_particle(np.array([x, y, 0.0]),
                                              np.array(e_initial_velocity), 
                                              e_charge,            
                                              e_mass,                 
                                              dt,               
                                              em_equations.Lorentz, #Landau_Lifshitz,   #   Lorentz
                                              em_equations.radiation_reaction, 
                                              push_method,                
                                              electric_field,                 
                                              magnetic_field)
                print(method_name)
                particle_manager.particles[ len(particle_manager.particles)-1 ].thickness=thickness
            else:
                counter += 1

    alpha = 0.01
    if counter1 % 10 == 0:
        total_field = np.zeros_like(E_field)
        for particle in particle_manager.particles:
            prtcl_pos = particle.position
            particle_field = electric_field_point(x_in, y_in, 0, prtcl_pos)
            magnitudes = np.linalg.norm(particle_field, axis=0)
            magnitudes = particle_field / (1 + alpha * magnitudes)
            total_field += magnitudes
        if len(particle_manager.particles)==num_prticle:
            E_transformed, B_transformed = em_equations.lorentz_transform_fields( E_field, B_field, particle_manager.particles[counter % num_prticle].velocity ) 
        else:
            n_stop = len(particle_manager.particles)

            E_transformed, B_transformed = em_equations.lorentz_transform_fields( E_field, B_field, np.array([0.0,0.0,0.0]) )

        def normalize_field(field):
            field = np.where(np.isfinite(field), field, 0)
            max_value = np.max(field)
            return field / (max_value + epsilon) if max_value != 0 else np.zeros_like(field)

        B_transformed = normalize_field(B_transformed)
        E_transformed = normalize_field(E_transformed)

    screen.fill((0, 0, 0))
    electric_field_surface.fill((0, 0, 0, 0))
    particle_manager.draw_arrows(  electric_field_surface,
                                 E_transformed,
                                 B_transformed,
                                 WIDTH,
                                 HEIGHT,
                                 GRID_SPACING,
                                 SCALE_FACTOR,
                                 ARROW_COLOR_E,
                                 ARROW_COLOR_B    )
    chosen_color = particle_manager.particles[counter % len(particle_manager.particles)].color
    pygame.draw.circle(screen, chosen_color, (WIDTH - 200, HEIGHT - 100), 10)
    particle_manager.update(electric_field, magnetic_field)
    # After particle_manager.update(...)zyzy
    if len(particle_manager.particles) > 1:
        n_stop = len(particle_manager.particles)
        screen_distance = 2 * max(WIDTH, HEIGHT)  # This might be better defined once outside the loop if it doesn't change
        # Filter out particles that have moved beyond the screen_distance
        particle_manager.particles = [
            particle for particle in particle_manager.particles
            if np.linalg.norm(particle.position[:2]) <= screen_distance
        ]
    particle_manager.draw(screen)
    screen.blit(electric_field_surface, (0, 0))
    pygame.display.flip()
    clock.tick(240)

pygame.quit()

pygame-ce 2.4.1 (SDL 2.28.5, Python 3.11.5)


In [ ]:
#comparing the simulation to the analytical solution of a moving charge partical on a constant magnetic field. 
# Set up the initial conditions
x0, y0, z0 = 0, 0, 0
v0 = 1e6  # Initial velocity magnitude (m/s)
alpha = 0 #np.pi / 4  # Angle between initial velocity and x-y plane (45 degrees)
q = 1 #1.60217663e-19  # Charge of the particle (C)
m = 1e-7 #1e-28 #9.10938356e-31  # Mass of the particle (kg)
dt = 1e-9  # Time step (s)
B = 0.1   # Magnetic field strength (T)
 


# Calculate the cyclotron frequency
omega = (q * B) / m

# Set the initial velocity components
vx0 =  v0 * np.cos(alpha)
vy0 = v0 * np.sin(alpha)
vz0 =0

# Create a particle with the initial conditions
particle = Particle(x0, y0, z0, q, m)
particle.velocity = np.array([vx0, vy0, vz0])
particle.dt= dt

# Define the magnetic field function
def magnetic_field_function(x, y, z):
    return np.array([0, 0, B])

# Define the electric field function
def electric_field_function(x, y, z):
    return np.array([0, 0, 0])

# Simulate the particle motion for a given time
time_steps = 6000
positions = []
positions.append([particle.x, particle.y, particle.z])
for _ in range(time_steps):
    particle.update(electric_field_function, magnetic_field_function, [])
    positions.append([particle.x, particle.y, particle.z])

# Convert positions to numpy array
positions = np.array(positions)

# Calculate analytical positions
t = np.arange(0, time_steps * dt, dt)
x_analytical = (vx0 / omega) * np.sin(omega * t)
y_analytical = (vx0 / omega) * (np.cos(omega * t) - 1)
z_analytical = vz0 * t

# Plot the simulated and analytical trajectories
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(positions[:, 0], positions[:, 1], positions[:, 2], label='Simulated')
ax.plot(x_analytical[0:], y_analytical[0:], z_analytical[0:], label='Analytical')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
plt.show()